The purpose of this notebook is to scrape the-numbers.com for all weekly box office data back to the beginning of 2010.
It is on a weekly basis Fri-Thursday in https://www.the-numbers.com/box-office-chart/weekly/2021/08/06#box_office_weekly_table=od14 with all the columns.  
Steps.  
Test parsing with Beautiful Soup for the current week
Generate dates back to December 18, 2009 2010/01/01 creating strings for all the reports
iterate through all the weeks.  will need to pause between weeks delay 5 or 10 secs or random number mean of 2 secs maybe?


In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime as DateTime, timedelta

In [2]:
def getdffromUrl(url):
    '''
    This getdffromUrl function returns a dataframe from the callung url. 

    :param url: The url to create a BeautifulSoup object in turn parsing the 
     report table and returning a dataframe.
    :type str: 
    
    :returns: A pandas dataframe of the report table from the-numbers
    :rtype: pandas.df
    '''
    
    html_page = requests.get(url) # Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing
    
    table_container = soup.find('table', id="box_office_weekly_table")
    #print(table_container)
    table_headers = table_container.findAll('th')
    table_rows = table_container.findAll('tr')
    
    weekdate = getdate_from_url(url)
    
    return parse_table(table_headers,table_rows, weekdate)

In [3]:
#https://www.the-numbers.com/box-office-chart/weekly/2021/08/06#box_office_weekly_table=od14
def buildurl(date):
    
    endurl = ''
    base_url ='https://www.the-numbers.com/box-office-chart/weekly/'
    suffix = '#box_office_weekly_table=od14'
    endurl = base_url + date + suffix
    print(endurl)
    
    return endurl

In [4]:
def getdate_from_url(url):
    date = ''
    date = url[52:62]
    print(date)
    return date

In [5]:
# Final Parse table from the numbers returns a data frame
def parse_table(table_headers, table_rows, weekdate):
    '''
    This parse_table2 function returns a dataframe from the inputs of
    html table headers and html table rows. 

    :param table_headers: The table_headers of an html table 
    :type ResultSet: 
    :param table_rows: The table_rows of an html table 
    :type ResultSet: 
    
    :returns: A pandas dataframe of the report table from the-numbers
    :rtype: pandas.df
    '''
    headers = []
    movies = []

    # create list of headers
    for i in range(0, len(table_headers)):
        headers.append(table_headers[i].get_text())

    # create list of rows/datacells
    for i in range(1, len(table_rows)-2):
        tds = table_rows[i].findAll('td')
        movie_dict = {}
        for i in range(0, len(tds)):
            movie_dict['weekdate'] = weekdate
            movie_dict[table_headers[i].get_text()] = tds[i].get_text()
        movies.append(movie_dict)

    df = pd.DataFrame(movies)
    
    #df.drop(columns=headers[0], inplace=True)

    return df

In [25]:
#create date Strings for 7 days at a time starting on 12-18-2009
#generate strings in format YYYY/MM/DD 2021/08/06
#Steps Convert start date to datetime
#Create date String save to list
#add 7 days to datetime
#create date string save to list.
#do while date is <=endstring.  should have around 574 dates
def generate_date_str():
    start_date ='2009/12/18'
    end_date = '2021/08/13'
    date_time_str = start_date
    date_time_obj = DateTime.strptime(date_time_str, '%Y/%m/%d')
    datestrs_lst =[]
    for i in range(0,609):
        #print ("The date is",  date_time_obj.strftime("%Y/%m/%d"))
        datestrs_lst.append(date_time_obj.strftime("%Y/%m/%d"))
        #add 7 days
        date_time_obj = date_time_obj + timedelta(days=7)

    return datestrs_lst
    


In [19]:
import time
def scrape_all():
    num=0
    lst=generate_date_str()
    #columns=['weekdate', 'prevwkrank','title', 'distr', 'wkgross', 'pct_chg_lw','num_thtrs', 'num_thtrs_chg', 'perthtr', 'tot_gross', 'wk_of_rel']
    weekly_df = pd.DataFrame()
    for dt in lst:
        weekly_df =weekly_df.append(getdffromUrl(buildurl(dt)))
        weekly_df.to_csv('./data/weekly_bo_nums.csv', encoding='utf-8',mode='a')
        if num % 10 ==0:
            print(num)
            time.sleep(5)
            
        num+=1
    return weekly_df

In [32]:
#report_df = scrape_all()

In [28]:
report_df.rename(columns={report_df.columns[1]:'prevwkrank', 'Movie': 'nums_title','Distr':'distr',
                          'Gross':'wkgross','%LW':'pct_chg_lw','Thr':'num_thtrs',
                         'ThrChng':'num_thtrs_chg','PerThr':'perthtr','TotalGross':'cumtot_gross','W':'wk_in run',},inplace=True)



In [153]:
report_df.to_csv('./data/weekly_bo_nums_tidy.csv', encoding='utf-8')

In [71]:
report_df["nums_title"].replace({"13 Hours: The Secret …": "13 Hours"}, inplace=True)

In [60]:
report_df["distr"].replace({"Eros Ente…": "Eros Entertainment"}, inplace=True)

In [68]:
report_df.distr.unique()

array(['Big World Pictures', 'Paramount Pictures', 'Magnolia Pictures',
       'Magnent Pictures', 'Walt Disney', 'Sony Pictures',
       'Indican Pictures', 'Film Movement', 'Freestyle Releasing', 'A24',
       'Eros Entertainment', 'Zeitgeist', 'TriBeca F…', 'The Film …',
       'Cinema Pu…', 'Strand', 'Abramoram…', 'Roadside Attractions',
       'Weinstein Co.', 'Fox Searchlight', 'China Lio…', 'Shadow',
       'IFC Films', 'FUNimation', 'Universal', 'Music Box…', 'AMC Indep…',
       'Kino Lorber', 'Internati…', 'Lionsgate', 'First Run…',
       'Reel Diva…', 'Amazon St…', 'Monterey …', 'Warner Bros.',
       'Idyllic E…', 'Rialto Pi…', 'Alive Mind', '20th Century Fox',
       'Oscillosc…', 'Saban Films', 'Drafthous…', 'United Ar…',
       'Cinema Libre', 'Self Dist…', 'Pure Flix…', 'Focus Features',
       'Menemsha …', nan, 'Cinema Guild', 'Codeblack…', 'The H Col…',
       'Purdie Di…', 'Good Deed…', 'Entertain…', 'Broad Gre…',
       'Adopt Films', 'Moriah Films', 'Kino Inte…',

In [154]:
report_df.isna().sum()

weekdate         0
prevwkrank       0
nums_title       0
distr            0
wkgross          0
pct_chg_lw       0
num_thtrs        0
num_thtrs_chg    0
perthtr          0
cumtot_gross     0
wk_in run        0
dtype: int64

In [73]:
report_df.sort_values(by=['nums_title', 'weekdate'],ascending=[False,True],inplace=True)

In [38]:
report_df.drop(columns=['Unnamed: 0'], inplace=True)

In [81]:
report_df= pd.read_csv('./data/weekly_bo_nums_tidy.csv', encoding='utf-8')

In [152]:
report_df[report_df.nums_title=='Ága']

,weekdate,prevwkrank,nums_title,distr,wkgross,pct_chg_lw,num_thtrs,num_thtrs_chg,perthtr,cumtot_gross,wk_in run
52150,2019/08/30,N,Ága,Big World Pictures,$978,,1,,$978,$978,1
52228,2019/09/06,(83),Ága,Big World Pictures,"$3,618",+270%,1,n/c,"$3,618","$4,596",2
52327,2019/09/13,(70),Ága,Big World Pictures,"$3,829",+6%,1,n/c,"$3,829","$8,425",3
52449,2019/09/20,(83),Ága,Big World Pictures,$378,-90%,1,n/c,$378,"$8,803",4
52531,2019/09/27,(100),Ága,Big World Pictures,"$3,765",+896%,2,+1,"$1,883","$12,568",5
52624,2019/10/04,(81),Ága,Big World Pictures,"$4,311",+15%,3,+1,"$1,437","$16,879",6
52724,2019/10/11,(72),Ága,Big World Pictures,"$1,934",-55%,3,n/c,$645,"$18,813",7
52820,2019/10/18,(72),Ága,Big World Pictures,"$3,082",+59%,3,n/c,"$1,027","$21,895",8
52913,2019/10/25,(76),Ága,Big World Pictures,"$2,127",-31%,1,-2,"$2,127","$24,022",9
53012,2019/11/01,(75),Ága,Big World Pictures,"$1,258",-41%,1,n/c,"$1,258","$25,280",10


In [137]:
##dropna
report_df.dropna(inplace=True)

In [ ]:
#######Test for Bass Model
#clean$ and , 


In [149]:
groupby_title = report_df.groupby(by='nums_title',sort=False)

In [151]:
groupby_title['wkgross'].apply(list).tolist()

[['$978',
  '$3,618',
  '$3,829',
  '$378',
  '$3,765',
  '$4,311',
  '$1,934',
  '$3,082',
  '$2,127',
  '$1,258',
  '$2,791',
  '$836',
  '$2,642',
  '$596'],
 ['$25,237,750',
  '$11,087,137',
  '$5,287,161',
  '$2,256,298',
  '$638,708',
  '$193,068',
  '$53,991',
  '$144,300'],
 ['$10,169,018',
  '$4,691,828',
  '$2,048,690',
  '$596,868',
  '$193,643',
  '$99,957'],
 ['$12,046', '$7,067', '$2,767', '$3,222', '$2,664'],
 ['$837'],
 ['$3,742', '$1,974', '$1,999', '$1,885'],
 ['$92,615,126',
  '$71,150,371',
  '$53,643,912',
  '$38,529,901',
  '$25,720,172',
  '$17,584,274',
  '$10,580,850',
  '$8,687,883',
  '$6,435,501',
  '$4,066,449',
  '$3,686,089',
  '$2,343,117',
  '$1,386,037',
  '$1,211,837',
  '$917,970',
  '$1,529,653',
  '$462,677',
  '$303,428',
  '$217,299',
  '$124,232',
  '$60,416',
  '$11,054'],
 ['$18,218,011',
  '$6,892,458',
  '$2,843,998',
  '$483,571',
  '$257,766',
  '$123,027',
  '$29,862'],
 ['$30,052,466',
  '$20,451,734',
  '$14,026,640',
  '$6,913,924',
  